# Explore the Cyber Ontology

This notebook uses introspection to explore the cyber graph ontology / schema which underpins Ontolocy.

In [ ]:
# Install dependencies if needed

%pip install neontology
%pip install ontolocy
%pip install networkx
%pip install ipysigma
%pip install yfiles_jupyter_graphs

In [ ]:
from ontolocy import *
from neontology import init_neontology
from neontology.utils import get_node_types, get_rels_by_type
import networkx as nx
from ipysigma import Sigma
from IPython.display import display, Markdown

# proprietary graph visualization library
from yfiles_jupyter_graphs import GraphWidget

In [ ]:
# if running on google colab, enable custom widgets
try:
    import google.colab
    from google.colab import output

    output.enable_custom_widget_manager()

except:
    pass

In [ ]:
def generate_schema_graph():
    node_types = get_node_types()

    nodes = [
        {
            "id": x.__primarylabel__,
            "name": x.__primarylabel__,
            "fields": list(x.model_fields.keys()),
        }
        for x in node_types.values()
    ]

    rel_types = get_rels_by_type()

    links = [
        {
            "source": x.source_class.__primarylabel__,
            "target": x.target_class.__primarylabel__,
            "link_label": f"{x.relationship_class.__relationshiptype__}: Relationship Type",
        }
        for x in rel_types.values()
        if hasattr(x.source_class, "__primarylabel__")
        and hasattr(x.target_class, "__primarylabel__")
        and hasattr(x.relationship_class, "__relationshiptype__")
        and x.source_class.__primarylabel__ in node_types.keys()
        and x.target_class.__primarylabel__ in node_types.keys()
    ]

    return {"edges": links, "nodes": nodes}

In [ ]:
def prepare_nx(node_link_data):
    return nx.node_link_graph(
        node_link_data,
        edges="edges",
    )


def display_graph_sigma(input_data):
    nx_graph = prepare_nx(input_data)

    return Sigma(
        nx_graph,
        node_label="name",
        node_color="name",
        default_edge_type="curve",
        node_border_color_from="node",
        show_all_labels=True,
    )


def display_graph_yfiles(input_data):
    nx_graph = prepare_nx(input_data)

    gw = GraphWidget(graph=nx_graph)

    gw.node_label_mapping = "name"
    gw.edge_label_mapping = "link_label"

    return gw

In [ ]:
input_data = generate_schema_graph()

In [ ]:
# Explore the schema with yfiles

display_graph_yfiles(input_data)

In [ ]:
# Uncomment below to explore the schema in Sigma

# display_graph_sigma(input_data)

In [ ]:
# Neontology nodes have a built in method which provides schema information

# Here we generate a simple table with the properties for a node

label_to_inspect = IPAddressNode

display(Markdown(label_to_inspect.neontology_schema().md_node_table()))

In [ ]:
# We can also see information about outgoing relationships

display(Markdown(label_to_inspect.neontology_schema().md_rel_tables()))